# **XGBoost**
The XGBoost (eXtreme Gradient Boosting) algorithm is a powerful, optimized, and scalable open-source machine learning library that implements gradient boosted decision trees. It is a leading method for supervised learning on structured/tabular data, known for high performance in both speed and predictive accuracy. XGBoost is an ensemble learning method that builds models sequentially, with each new, simple decision tree (a "weak learner") attempting to correct the prediction errors (residuals) of all previous trees combined. This process is formalized as a gradient descent optimization problem to minimize a specified loss function. 

## The working (Classification)

- #### Get a base prediction
The first step is to get a base prediction for our dataset, in terms of probability and log(odds).

$$ p = \text{Base probablity for all data points} $$
$$ log(odds) = log_{e}(\frac {p}{1-p}) $$

- #### Calculate residuals
The next step is to calculate residual for each data point.

$$ r_{i} = y_{i} - p $$

- #### Make the tree on the basis of Gain
The first decision tree is created and we use **Gain** to determine the best split. The split havibe the highest **gain** will be selected. The Tree will be trained on the data $ \{x, r \} $.

$$ Gain = S_{\text{left subtree}} + S_{\text{right subtree}} - S_{\text{Root}} $$
$$ \text{S here is the similarity value of a root or leaf node} $$
$$ S_{tree} = \frac {(\sum_{i=1}^{n} r_{i})^2}{(\sum_{i=1}^{n} p(1-p)) + λ} $$
$$ n = \text{number of values in the root or leaf} $$
$$ p = \text{initial probability} $$
$$ λ = \text{A hyperparameter to reduce similarity score} $$

- #### Make prediction
For any new data, we will calculate the value from this formula

$$ v = \text{initial log(odds)} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Similarity value of leaf node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

We will then apply a sigmoid activation function on this to get the value in range of 0 to 1.

$$ p = σ(v) = \frac {1}{1 - e^v} $$

- #### Repeat
Repeat all the process again until we are done with making the predefined number of trees.
The final output will be derived by the following formula:

$$ p = σ(v) = \frac {1}{1 - e^v} $$
$$ v = \text{initial log(odds)} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Similarity value of leaf node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

## The working (Regression)

- #### Get a base prediction
The first step is to get a base prediction for our dataset. The initial prediction is $0.5$ in most of cases

$$ p = \text{Mean of all the output points} $$

- #### Calculate residuals
The next step is to calculate residual for each data point.

$$ r_{i} = y_{i} - p $$

- #### Make the tree on the basis of Gain
The first decision tree is created and we use **Gain** to determine the best split. The split havibe the highest **gain** will be selected. The Tree will be trained on the data $ \{x, r \} $.

$$ Gain = S_{\text{left subtree}} + S_{\text{right subtree}} - S_{\text{Root}} $$
$$ \text{S here is the similarity value of a root or leaf node} $$
$$ S_{tree} = \frac {(\sum_{i=1}^{n} r_{i})^2}{n + λ} $$
$$ n = \text{number of values in the root or leaf} $$
$$ λ = \text{A hyperparameter to reduce similarity score} $$

- #### Make prediction
For any new data, we will calculate the value from this formula

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Similarity value of leaf node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

We will then apply a sigmoid activation function on this to get the value in range of 0 to 1.

- #### Repeat
Repeat all the process again until we are done with making the predefined number of trees.
The final output will be derived by the following formula:

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Similarity value of leaf node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

## How XGBoost does regularization
XGBoost minimizes the Overfitting of the model by regularisation. It simply adds a hypermeter (`λ`) in the denominator that helps reducing the similarity score of the leaf.

### Pruning
Tree pruning is done to reduce the overfitting conditions. We compare the Gain of the split with a predefined hyperparameter (`γ`).

$$
\begin{equation}
\text{Gain} - γ = 
\begin{cases}
\text{If positive, then do not prune.} \\
\text{If negative, then prune.}
\end{cases}
\end{equation}
$$

When we have higher `λ` then the gain reduces and it is more likely for that split to get pruned. However, if we make a very large value of `λ` then tree can be pruned to much.\
The values of `λ` and `γ` needs to determined properly


## Implementation

In [1]:
# Get the data set

from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
print(data['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [2]:
import pandas as pd

X, y = pd.DataFrame(data['data'], columns=data['feature_names']), data['target']
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# split into train test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
# train the model
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [8]:
# predict
y_pred = xgb.predict(X_test)
y_pred

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1])

In [17]:
# check for accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_true=y_test, y_pred=y_pred))
print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(classification_report(y_true=y_test, y_pred=y_pred))

0.965034965034965
[[51  3]
 [ 2 87]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        54
           1       0.97      0.98      0.97        89

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.97      0.97      0.96       143

